In [1]:
!pip3 install numpy

In [2]:
# python3.5 buildtagger.py <train_file_absolute_path> <model_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def train_model(train_file, model_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    train_file = sys.argv[1]
    model_file = sys.argv[2]
    start_time = datetime.datetime.now()
    train_model(train_file, model_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


Finished...
Time: 0:00:00.000111


In [3]:
# python3.5 runtagger.py <test_file_absolute_path> <model_file_absolute_path> <output_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def tag_sentence(test_file, model_file, out_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    test_file = sys.argv[1]
    model_file = sys.argv[2]
    out_file = sys.argv[3]
    start_time = datetime.datetime.now()
    tag_sentence(test_file, model_file, out_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


IndexError: list index out of range

## Creating the model statistics

Need to handle special cases like these:
 - fractions like: 1/2/CD
 - multiple listings: pianist/bassoonist/composer/NN
 
 As far as I understand all of the words belong to the same POS. 
 A function that adds both words should be implemented, but an initial version function should simply take the last two elements and uses them as word + tag.
 
 
 Also, likely the score could be improved by adding add-one-smoothing

## Using only tags for dictionary, but still using touples instead of strings

In [4]:
import itertools
import _pickle as pickle

flatten = lambda x: list(itertools.chain.from_iterable(x))

class Dictionary():
    def __init__(self):
        self.word_dict = {}
        self.tag_dict = {}
        
        self.word_count = 0
        self.tag_count = 0
    
    def add_word(self, word):
        if word not in self.word_dict.values():
            self.word_dict[self.word_count] = word
            self.word_count += 1
            return self.word_count - 1
        return self.get_word_id(word)
    
    def add_tag(self, tag):
        if tag not in self.tag_dict.values():
            self.tag_dict[self.tag_count] = tag
            self.tag_count += 1
            return self.tag_count - 1
        return self.get_tag_id(tag)
            
    def get_word_id(self, word):
        for key, known_word in self.word_dict.items():
            if word == known_word:
                return key
    
    def get_tag_id(self, tag):
        for key, known_tag in self.tag_dict.items():
            if tag == known_tag:
                return key
    
    def get_word_tag_tuple(self, word_tag):
        words_tag = word_tag.split("/")
        assert len(words_tag) > 1, "Bad format for word/tag: {}".format(word_tag)
        
        word_ids = [self.get_word_id(word) for word in words_tag[:-1]]
        tag_id = self.get_tag_id(words_tag[-1])
        word_ids.append(tag_id)
        return tuple(word_ids)
    
    
class PosStatsText():
    def __init__(self):
        self.emissions = {}
        self.transitions = {}
        self.unique_words = set()
        self.dictionary = Dictionary()
        
    def add_tag(self, tag):
        return self.dictionary.add_tag(tag)
        
    def add_emission(self, word_id, tag_id):
        if (word_id, tag_id) not in self.emissions.keys():
            self.emissions[(word_id, tag_id)] = 1
        else:
            self.emissions[(word_id, tag_id)] += 1
        
    def add_transition(self, previous_tag_id, tag_id):
        if (previous_tag_id, tag_id) not in self.transitions.keys():
            self.transitions[(previous_tag_id, tag_id)] = 1
        else:
            self.transitions[(previous_tag_id, tag_id)] += 1

            
def extract_frequencies_tuple_text(filename):    
    pos_stats = PosStatsText()

    with open(filename) as training_file:
        word_tag_pairs = flatten([sentence.split(" ") for sentence in training_file.read().split("\n")])        
        previous_tag_id = pos_stats.add_tag(word_tag_pairs[0].split("/")[-1])

        for word_tag in word_tag_pairs:
            if len(word_tag.split("/")) < 2:
                continue

            word, tag = word_tag.split("/")[-2:]
            tag_id = pos_stats.add_tag(tag)
            
            pos_stats.add_emission(word, tag_id)
            pos_stats.add_transition(previous_tag_id, tag_id)

            previous_tag_id = tag_id
    
    return pos_stats 

def write_output(model_stats, filename="model-file"):
    with open(filename, "wb") as model_file:
        model_file.write(pickle.dumps(model_stats))
    
def train_model(train_file, model_file):
    model_stats = extract_frequencies_tuple_text(train_file)
    write_output(model_stats, filename=model_file)

In [10]:
model_stats = extract_frequencies_tuple_text('sents.train')

In [20]:
unique_words = set()
[word[0] for word in model_stats.emissions.keys()]

['In',
 'an',
 'Oct.',
 '19',
 'review',
 'of',
 '``',
 'The',
 'Misanthrope',
 "''",
 'at',
 'Chicago',
 "'s",
 'Goodman',
 'Theatre',
 '(',
 'Revitalized',
 'Classics',
 'Take',
 'the',
 'Stage',
 'in',
 'Windy',
 'City',
 ',',
 'Leisure',
 '&',
 'Arts',
 ')',
 'role',
 'Celimene',
 'played',
 'by',
 'Kim',
 'Cattrall',
 'was',
 'mistakenly',
 'attributed',
 'to',
 'Christina',
 'Haag',
 '.',
 'Ms.',
 'plays',
 'Elianti',
 'Rolls-Royce',
 'Motor',
 'Cars',
 'Inc.',
 'said',
 'it',
 'expects',
 'its',
 'U.S.',
 'sales',
 'remain',
 'steady',
 'about',
 '1,200',
 'cars',
 '1990',
 'luxury',
 'auto',
 'maker',
 'last',
 'year',
 'sold',
 '1,214',
 'Howard',
 'Mosher',
 'president',
 'and',
 'chief',
 'executive',
 'officer',
 'he',
 'anticipates',
 'growth',
 'for',
 'Britain',
 'Europe',
 'Far',
 'Eastern',
 'markets',
 'BELL',
 'INDUSTRIES',
 'increased',
 'quarterly',
 '10',
 'cents',
 'from',
 'seven',
 'a',
 'share',
 'new',
 'rate',
 'will',
 'be',
 'payable',
 'Feb.',
 '15',
 'A'

In [5]:
%%time
train_model('sents.train', 'model-file')

CPU times: user 5.53 s, sys: 91.2 ms, total: 5.62 s
Wall time: 5.8 s


# Simple implementation of POS tagger

In [143]:
def read_and_tag(input_filename, pos_tagger):
    tagged_sent = []

    with open(input_filename, "r") as input_file:
        sentences = input_file.read().split("\n")

        for sentence in progress_bar(sentences):
            tagged_sent.append(pos_tagger(sentence))
    
    return tagged_sent

## Problems

How do I deal with words that have yet to be seen in the training data?
- Possibly remove capitalization before collecting stats?

Very slow performance
- Create a dictionary of words and use the word embeddings instead of a dictionary

Need to calculate the begining of the sentence transition as well

Possibly ignore dictionary creation, but use tuples instead of the initial implementation of the training algorithm

In [154]:
from functools import partial
from fastprogress import progress_bar

common_tag = "NN"

def word_emissions(word, keys):
    return [key for key in keys if key.startswith(word)]

def simple_pos_tagger(sentence, stats):
    tagged_sentence = []
    keys = stats.emissions.keys()
    
    for word in sentence.split(" "):
        emission_tags = word_emissions(word, keys)
        emission_probs = [stats.emissions[word_tag] for word_tag in emission_tags]

        if len(emission_tags) > 0:
            tagged_sentence.append(emission_tags[np.argmax(emission_probs)])
        else:
#             print("Word hasn't appeared in the training data: {}".format(word))
            tagged_sentence.append("{}/{}".format(word, common_tag))
        
    return " ".join(tagged_sentence)

simple_stats_tagger = partial(simple_pos_tagger, stats=loaded)

In [156]:
%%time

tagged = read_and_tag("sents.test", pos_tagger=simple_stats_tagger)

CPU times: user 9min 7s, sys: 3.54 s, total: 9min 11s
Wall time: 9min 32s


In [160]:
with open("simple_test_output.txt", "w") as out_file:
    out_file.write("\n".join(tagged[:-1]))

# Hidden Markov Model implemented with Viterbi algorithm

In [8]:
from functools import partial


class PosStatsModel():
    def __init__(self, pos_stats=None):
        self.emissions = {}
        self.transitions = {}
        self.dictionary = Dictionary()
        self.MIN_PROB_VALUE = 10e-5
        
        if pos_stats is not None:
            self.emissions = pos_stats.emissions
            self.transitions = pos_stats.transitions
            self.dictionary = pos_stats.dictionary
        
    def add_tag(self, tag):
        return self.dictionary.add_tag(tag)
    
    def get_all_tag_ids(self):
        return self.dictionary.tag_dict.keys()
        
    def add_emission(self, word_id, tag_id):
        if (word_id, tag_id) not in self.emissions.keys():
            self.emissions[(word_id, tag_id)] = 1
        else:
            self.emissions[(word_id, tag_id)] += 1
        
    def add_transition(self, previous_tag_id, tag_id):
        if (previous_tag_id, tag_id) not in self.transitions.keys():
            self.transitions[(previous_tag_id, tag_id)] = 1
        else:
            self.transitions[(previous_tag_id, tag_id)] += 1
    
    def get_emission_prob(self, word, tag):
        try:
            return self.emissions[(word, tag)]
        except KeyError:
            return self.MIN_PROB_VALUE
        
    def get_transition_prob(self, prev_tag, tag):
        try:
            return self.transitions[(prev_tag, tag)]
        except KeyError:
            return self.MIN_PROB_VALUE
    
    
def get_transition_probs(previous_probs, pos_stats, tags):
    max_ids = []
    probs = []
    
    for tag_id in tags:
        transitions = [pos_stats.get_transition_prob(prev_tag_id, tag_id) for prev_tag_id in tags]
        transitions_with_previous = previous_probs * np.array(transitions)
        max_id = np.argmax(transitions_with_previous)
        max_ids.append(max_id)
        probs.append(transitions_with_previous[max_id])
        
    return probs, max_ids

def combine_words_tags(words, tag_list, pos_stats):
    for i in range(len(words)):
        words[i] = words[i] + "/" + pos_stats.dictionary.tag_dict[tag_list[i]]
    
    return " ".join(words)
    
def get_sentence_tags(sentence, pos_stats):
    pos_tags = pos_stats.get_all_tag_ids()
    words = sentence.split(" ")

    viterbi = np.zeros((len(pos_tags), len(words)), float)
    backpointers = np.zeros((len(pos_tags), len(words)), int)

    # fill in initial probs
    # for now skipping the beginning of the sentence probability

    viterbi[:, 0] = [pos_stats.get_emission_prob(words[0], tag_id) for tag_id in pos_tags]
    backpointers[:, 0] = -1

    # fill in the full matrix
    for i, word in enumerate(words[1:]):
        emission_probs = [pos_stats.get_emission_prob(word, tag_id) for tag_id in pos_tags]
        transition_probs, new_backpointers = get_transition_probs(viterbi[:, i], pos_stats, pos_tags)

        viterbi[:, i + 1] = np.array(emission_probs) * np.array(transition_probs)
        backpointers[:, i + 1] = new_backpointers

    # Also need to calculate the ending probabilities
    tag_list = np.zeros(len(words) + 1, int)
    
    final_tag = np.argmax(viterbi[:, -1]) # this should be in accordance to the end of the sentence
    tag_list[-1] = final_tag
    previous_back = final_tag

    for word_pos in reversed(range(1, len(words))):
        tag_list[word_pos] = backpointers[previous_back, word_pos]
        previous_back = tag_list[word_pos]
        
    tag_list = tag_list[1:]
        
    return combine_words_tags(words, tag_list, pos_stats)

def read_and_tag(input_filename, pos_tagger):
    tagged_sent = []

    with open(input_filename, "r") as input_file:
        sentences = input_file.read().split("\n")

        for sentence in sentences:
            tagged_sent.append(pos_tagger(sentence))
    
    return tagged_sent

def load_model_file(filename):
    with open(filename, "rb") as model_read:
        model_file = pickle.load(model_read)
        return model_file
    
def tag_sentence(test_file, model_file, out_file):
    model_stats = PosStatsModel(pos_stats=load_model_file(model_file))
    hmm_viterbi_tagger = partial(get_sentence_tags, pos_stats=model_stats)
    
    tagged_sentences = read_and_tag(test_file, pos_tagger=hmm_viterbi_tagger)
    with open(out_file, "w") as out_file:
        out_file.write("\n".join(tagged_sentences[:-1]))
        
    print('Finished...') 

## Implementing the new prediction model

In [9]:
%%time

tag_sentence("sents.test", "model-file", "viterbi-preds.txt")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: overflow encountered in multiply


Finished...
CPU times: user 1min 23s, sys: 862 ms, total: 1min 24s
Wall time: 1min 28s
